<a href="https://colab.research.google.com/github/rgprado/processamento_larga_escala/blob/main/dask_assignment_RodrigoPrado_better.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sobre os dados

O arquivo CSV contém eventos 'click' ou 'view' no tempo, de usuários em anúncios de determinadas campanhas.

**Descrição das colunas:**  
timestamp,user_id,action,adId,campaignId 

**Amostra:**  
2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01  
2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02  
2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02  
2016-03-26 06:27:00,78aa2467-b502-413b-94e9-04ec8210bd13,click,adId_07,campaignId_03

**Nome do arquivo CSV:**  
data/ad_action.csv

## Sobre as questões

As questões devem ser respondidas usando a tecnologia Dask.

Quando utilizar pandas tenha cuidado para evitar estouro de memória, sempre imaginado que vai executar o código com uma grande massa de dados.

Mesmo que não consiga terminar alguma questão, favor enviar, porque parte do código pode valer alguma pontuação.

## Processamento Dados em Larga Escala

**Professor: Anderson Neves**

**Alunos: Rodrigo Prado e Rivaldo Junior**

In [ ]:
import pandas as pd
import numpy as np

import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()
client = Client(cluster)
client

/usr/local/lib/python3.9/dist-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40743 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:40743/status,
Dashboard: http://127.0.0.1:40743/status,Workers: 2
Total threads: 2,Total memory: 12.68 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43479,Workers: 2
Dashboard: http://127.0.0.1:40743/status,Total threads: 2
Started: Just now,Total memory: 12.68 GiB
Comm: tcp://127.0.0.1:41479,Total threads: 1
Dashboard: http://127.0.0.1:42953/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:34873,


In [ ]:
df = dd.read_csv('ad_action.csv', header=None, parse_dates=[0])
df.columns = ['timestamp','user_id','action','adId','campaignId']
df

,timestamp,user_id,action,adId,campaignId
npartitions=1,,,,,
,datetime64[ns],object,object,object,object
,...,...,...,...,...


In [ ]:
df.head(5)

,timestamp,user_id,action,adId,campaignId
0,2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01
1,2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02
2,2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02
3,2016-03-26 06:27:00,78aa2467-b502-413b-94e9-04ec8210bd13,click,adId_07,campaignId_03
4,2016-01-02 04:57:00,fef9a98c-d73e-48ef-b2cb-766ba85dc3e3,view,adId_02,campaignId_02


## 1) Quais são as top 3 campanhas que geraram mais eventos? Ordene pela quantidade de eventos (2,5 pontos)

In [ ]:
%%time
most_popular_cp = df[['campaignId', 'timestamp']].groupby('campaignId')\
    ['timestamp']\
    .count()\
    .nlargest(3)\
    .compute()

print('Top 3 campanhas com mais enventos:', most_popular_cp, '\n')

Top 3 campanhas com mais enventos: campaignId
campaignId_02    91216
campaignId_03    87036
campaignId_01    76461
Name: timestamp, dtype: int64 

CPU times: user 165 ms, sys: 14.4 ms, total: 179 ms
Wall time: 1.13 s


## 2) Qual campanha teve mais clicks? (2,5 pontos)

In [ ]:
%%time
most_click_campain = df[df['action'] == 'click'][['campaignId', 'action']].groupby('campaignId')\
    ['action']\
    .count()\
    .nlargest(1)\
    .compute()\
    .index[0]

print('Campanha com mais clicks:', most_click_campain, '\n')

Campanha com mais clicks: campaignId_02 

CPU times: user 177 ms, sys: 20.1 ms, total: 197 ms
Wall time: 1.17 s


In [ ]:
%%time
most_click_campain = df[['campaignId', 'action']]\
    .where(df['action'] == "click")\
    .groupby('campaignId')\
    ['action']\
    .count()\
    .nlargest(1)\
    .compute()\
    .index[0]

print('Campanha com mais clicks:', most_click_campain, '\n')

Campanha com mais clicks: campaignId_02 

CPU times: user 155 ms, sys: 22.6 ms, total: 178 ms
Wall time: 1.06 s


## 3) Existe sazonalidade mensal? Qual mês teve o maior total de eventos acumulado? (2,5 pontos)

In [ ]:
%%time 

most_event_month = df[['timestamp', 'action']]\
    .assign(month=lambda x: x['timestamp'].dt.month_name())\
    .groupby('month')\
    ['timestamp']\
    .count()\
    .nlargest(1)\
    .compute()\
    .index[0]

print('Mês teve o maior total de eventos acumulado:', most_event_month, '\n')

Mês teve o maior total de eventos acumulado: January 

CPU times: user 130 ms, sys: 16.5 ms, total: 147 ms
Wall time: 766 ms


## 4) Nas situações onde existe um evento de view seguido de um evento de click criados pelo mesmo usuário no mesmo anúncio e campanha, quais são os 5 pares de anúncio e campanha com menores médias de tempo entre os dois eventos (2,5 pontos)

In [ ]:
# Verificando se tem algum usuário que só clicou ou visualizou 
clicks_df = df[['user_id', 'action']]\
    [df['action'] == 'click']\
    .groupby("user_id")\
    .count()\
    .rename(columns={'action': 'n_clicks'})

views_df = df[['user_id', 'action']]\
    [df['action'] == 'view']\
    .groupby("user_id")\
    .count()\
    .rename(columns={'action': 'n_views'})

actions_df = clicks_df.merge(views_df, on='user_id', how='outer').fillna(0)
filtro = actions_df.loc[(actions_df['n_views'] == 0) | (actions_df['n_clicks'] == 0)]

In [ ]:
# a ideia é remover todos os registros de usuario que só tenham view ou click
series_to_match = df[filtro].compute() 
df = df[df['user_id'].isin(series_to_match) == False]


In [ ]:
# ordenar o dataframe por timestamp para ter os eventos em sequencia
df = df.sort_values(['timestamp'], ascending=True).reset_index(drop=True)

In [ ]:
df.head()

,timestamp,user_id,action,adId,campaignId
0,2016-01-01 00:00:00,c577e717-1be3-4e6a-abc0-dd9350a6be72,click,adId_06,campaignId_02
1,2016-01-01 00:07:00,4fde5dfa-ccde-4c2e-9038-870a21956725,view,adId_07,campaignId_02
2,2016-01-01 00:07:00,0fb092f2-6721-4dd9-b574-0dfe088a2adf,click,adId_10,campaignId_03
3,2016-01-01 00:11:00,2dbd9392-768e-4ba5-9c07-067733421d74,click,adId_03,campaignId_03
4,2016-01-01 00:12:00,cd3e0222-0796-450a-9a84-df007635b01d,click,adId_09,campaignId_01


In [ ]:
# criando um dataframe dando shift de uma linha
df_shift = df.shift(-1)
df_shift.columns = ['timestamp2', 'user_id2', 'action2', 'adId2', 'campaignId2']

In [ ]:
# concatenando o shift dataframe com o dataframe
# terei a informação a linha atual e a próxima em uma unica linha
%%time 
df = dd.concat([df, df_shift], axis=1)

CPU times: user 5.27 ms, sys: 2.11 ms, total: 7.38 ms
Wall time: 15.5 ms


/usr/local/lib/python3.9/dist-packages/dask/dataframe/multi.py:1222: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [ ]:
# Filtrando todos os registros que tem o match descrito na questão, click depois de view e mesmo user, propaganda e campanha.
df_pares = df[
    (df['user_id'] == df['user_id2']) &
    (df['action'] == 'view') & (df['action2'] == 'click') &
    (df['adId'] == df['adId2']) & 
    (df['campaignId'] == df['campaignId2']) 
]

In [ ]:
# criando uma coluna com a diferença entre o view e click
df_pares['tempo_eventos'] = (df_pares['timestamp2'] - df_pares['timestamp']).dt.seconds

In [ ]:
df_pares.head()

,timestamp,user_id,action,adId,campaignId,timestamp2,user_id2,action2,adId2,campaignId2,tempo_eventos
12331,2016-01-12 07:21:00,796ea1b7-7b9c-4e08-bc3c-1a01bd246869,view,adId_02,campaignId_01,2016-01-12 07:32:00,796ea1b7-7b9c-4e08-bc3c-1a01bd246869,click,adId_02,campaignId_01,0 days 00:11:00
15937,2016-01-17 10:06:00,1d625585-afe1-44e5-b072-b42a9b03a9ee,view,adId_04,campaignId_02,2016-01-17 10:06:00,1d625585-afe1-44e5-b072-b42a9b03a9ee,click,adId_04,campaignId_02,0 days 00:00:00
27068,2016-02-02 20:45:00,33a9fa72-3c64-4daf-962f-f4e6a69df8c1,view,adId_03,campaignId_02,2016-02-02 20:46:00,33a9fa72-3c64-4daf-962f-f4e6a69df8c1,click,adId_03,campaignId_02,0 days 00:01:00
33868,2016-02-12 21:31:00,bc72b04a-0e57-40d9-9a1b-42523561de50,view,adId_09,campaignId_02,2016-02-12 21:32:00,bc72b04a-0e57-40d9-9a1b-42523561de50,click,adId_09,campaignId_02,0 days 00:01:00
37548,2016-02-18 10:11:00,8b16f80d-7607-4907-b125-5aceddee7fd0,view,adId_08,campaignId_03,2016-02-18 10:11:00,8b16f80d-7607-4907-b125-5aceddee7fd0,click,adId_08,campaignId_03,0 days 00:00:00


In [ ]:
# Calculando as menores médias das propaganda e campanha
%%time
print('Os 5 pares de anúncio e campanha com menores médias de tempo entre os dois eventos:\n')

df_pares[['adId', 'campaignId', 'tempo_eventos']].groupby(['adId','campaignId'])\
    ['tempo_eventos']\
    .mean()\
    .nsmallest(5)\
    .compute()\
    .reset_index()



Os 5 pares de anúncio e campanha com menores médias de tempo entre os dois eventos:

CPU times: user 252 ms, sys: 45.8 ms, total: 297 ms
Wall time: 1.22 s


,adId,campaignId,tempo_eventos
0,adId_04,campaignId_02,0.0
1,adId_04,campaignId_03,0.0
2,adId_08,campaignId_03,0.0
3,adId_06,campaignId_02,20.0
4,adId_02,campaignId_02,30.0


# Média igual a zero

Achei estranho alguns pares aparecerem zerados. Mas de fatos temos pares de anuncio e campanha com a diferença entre eventos igual zero. 

In [ ]:
df_pares[(df_pares['adId'] == 'adId_04') & (df_pares['campaignId'] == 'campaignId_02')].compute()

,timestamp,user_id,action,adId,campaignId,timestamp2,user_id2,action2,adId2,campaignId2,tempo_eventos
15937,2016-01-17 10:06:00,1d625585-afe1-44e5-b072-b42a9b03a9ee,view,adId_04,campaignId_02,2016-01-17 10:06:00,1d625585-afe1-44e5-b072-b42a9b03a9ee,click,adId_04,campaignId_02,0
224595,2016-11-16 20:42:00,79eb3c4b-2c9b-4d77-81ac-0f7d30f5f024,view,adId_04,campaignId_02,2016-11-16 20:42:00,79eb3c4b-2c9b-4d77-81ac-0f7d30f5f024,click,adId_04,campaignId_02,0


In [ ]:
df_pares[(df_pares['adId'] == 'adId_04') & (df_pares['campaignId'] == 'campaignId_03')].compute()

,timestamp,user_id,action,adId,campaignId,timestamp2,user_id2,action2,adId2,campaignId2,tempo_eventos
245034,2016-12-16 09:48:00,7eb50763-1258-458a-a7e1-8dccd09759c5,view,adId_04,campaignId_03,2016-12-16 09:48:00,7eb50763-1258-458a-a7e1-8dccd09759c5,click,adId_04,campaignId_03,0


In [ ]:
df_pares[(df_pares['adId'] == 'adId_08') & (df_pares['campaignId'] == 'campaignId_03')].compute()

,timestamp,user_id,action,adId,campaignId,timestamp2,user_id2,action2,adId2,campaignId2,tempo_eventos
37548,2016-02-18 10:11:00,8b16f80d-7607-4907-b125-5aceddee7fd0,view,adId_08,campaignId_03,2016-02-18 10:11:00,8b16f80d-7607-4907-b125-5aceddee7fd0,click,adId_08,campaignId_03,0
